## 데커레이터는 함수가 함수를 한번 감싸서 리턴하는 형태이다.
### 1번과 2번은 동일하다

In [2]:
def decorate(func) :
    return func

#1
@decorate
def target() :
    print("running target()")
target()

running target()


In [3]:
    
#2
def target() :
    print("running target()")
target = decorate(target)
target()

running target()


## 데커레이터가 실행되는 시점
### 데커레이터는 import time 에 실행되며 이는 runtime 과 구분된다.

In [9]:
# registration.py 을 확인하고 콘솔에서 실행해보자
import registration

In [10]:
# import time에 데코레이트 함수가 실행되어 registry에 두 함수가 추가된 것을 확인할 수 있다.
registration.registry

[<function registration.f1()>, <function registration.f2()>]

## 데커레이터로 개선된 전략 패턴
### promos 에 리스트를 추가하는 것을 깜빡하였을 경우를 데커레이터를 활용해 해결한다.

In [ ]:
promos = []

# 데커레이트된 함수들은 import time에 실행되어 promos 에 추가될 것이다.
def promotion(promo_func) :
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity_promo(order) :
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0
@promotion
def bulk_item_promo(order) :
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart :
        if item.quantity >= 20 :
            discount += item.total() * 0.1
    return discount

@promotion
def large_order_promo(order) :
    '''10종류 이상의 상품을 구입하면 전체 7% 할인 적용'''
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10 :
        return order.total() * .07
    return 0

def best_promo(order) :
    """ 최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)